# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [24]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [25]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Cassandra tables

In [26]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [27]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling for Apache Cassandra

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [28]:
# Make a connection to a Cassandra instance in local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [29]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [30]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

With Apache Cassandra we model the database tables on the queries we want to run.

## Queries

In [31]:
# Use this dict to get column numbers of each column by label.
cols = {
    'artist':0,
    'firstName':1,
    'gender':2,
    'itemInSession':3,
    'lastName':4,
    'length':5,
    'level':6,
    'location':7,
    'sessionId':8,
    'song':9,
    'userId':10}

#### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [32]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# PRIMARY KEY: (sessionId, itemInSession) => uniquely identify each row in the table.
# PARTITION KEY: (sessionId, itemInSession) => determine the distribution of data across the system.
# CLUSTERING COLUMN: None

query = "CREATE TABLE IF NOT EXISTS Sessions "
query = query + "(sessionId INT, itemInSession INT, artist TEXT, song TEXT, length FLOAT,\
                 PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [33]:
# Read CSV and insert values into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Add values into Sessions table
        query = "INSERT INTO Sessions (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[cols['sessionId']]), int(line[cols['itemInSession']]), line[cols['artist']], line[cols['song']], float(line[cols['length']])))

#### Do a SELECT to verify that the data have been inserted into each table

In [34]:
# Verify the data was entered into the table
query = "SELECT artist, song, length FROM Sessions WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [35]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# PRIMARY KEY: ((userId, sessionId), itemInSession)) => uniquely identify each row in the table.
# PARTITION KEY: (userId, sessionId) => determine the distribution of data across the system.
# CLUSTERING COLUMN: itemInSession => sort the data in sorted ascending order.

query = "CREATE TABLE IF NOT EXISTS UserSessions "
query = query + "(userId INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, firstname TEXT, lastname TEXT, \
                 PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
               

In [36]:
# Read CSV and insert values into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Add values into UserSessions
        query = "INSERT INTO UserSessions (userId, sessionId, itemInSession, artist, song, firstname, lastname) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[cols['userId']]), int(line[cols['sessionId']]), int(line[cols['itemInSession']]), line[cols['artist']], line[cols['song']], line[cols['firstName']], line[cols['lastName']]))

In [37]:
# Verify the data was entered into the table
query = "SELECT artist, song, firstname, lastname FROM UserSessions WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, '|', row.song, '|', row.firstname, row.lastname)

Down To The Bone | Keep On Keepin' On | Sylvie Cruz
Three Drives | Greece 2000 | Sylvie Cruz
Sebastien Tellier | Kilometer | Sylvie Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz


#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [38]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# PRIMARY KEY: ((song), userId)) => uniquely identify each row in the table.
# PARTITION KEY: song => determine the distribution of data across the system.
# CLUSTERING COLUMN: userId => sort the data in sorted ascending order.

query = "CREATE TABLE IF NOT EXISTS UserSongs "
query = query + "(song TEXT, userId INT, firstname TEXT, lastname TEXT, PRIMARY KEY ((song), userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [39]:
# Read CSV and insert values into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Add values into UserSongs
        query = "INSERT INTO UserSongs (song, userId, firstname, lastname) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[cols['song']], int(line[cols['userId']]), line[cols['firstName']], line[cols['lastName']]))

In [40]:
# Verify the data was entered into the table
query = "SELECT firstname, lastname FROM UserSongs WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [41]:
try:
    session.execute("DROP TABLE IF EXISTS Sessions")
    session.execute("DROP TABLE IF EXISTS UserSessions")
    session.execute("DROP TABLE IF EXISTS UserSongs")
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [42]:
session.shutdown()
cluster.shutdown()